In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import statistics

from konlpy.tag import Twitter
from konlpy.utils import pprint
twitter = Twitter()
from collections import Counter
from collections import namedtuple

pd.set_option('max_colwidth', 500)

common_front = '../../dataset/go_0715/gogosing_' 
common_back = '.json'

def get_file(file_num):   
    file_path = common_front + str(file_num) + common_back
    json_data = open(file_path).read()
    data = json.loads(json_data)
    df_data = pd.DataFrame(data)
    return df_data


# 74번 파일이 에러가 나서 제외함
def concat_file(file_num):
    start_file = pd.DataFrame(get_file(1))
    for i in range(2,file_num+1):
        if( i == 74):
            continue
        df_tmp = pd.DataFrame(get_file(i))
        start_file = pd.concat([start_file, df_tmp])
    start_file = start_file.reset_index(drop=True)
    return start_file

In [3]:
go_data = concat_file(156)

# Practice

In [13]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from konlpy.tag import Twitter

/usr/local/lib/python2.7/dist-packages/scipy/sparse/lil.py:16: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:169: R

In [14]:
data = ["I love machine learning. Its awesome.",
       "I love coding in python",
       "I love building cahtbots",
       "they chat amagingly well"]

In [15]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

Training

In [16]:
max_epochs = 100
vec_size = 200
alpha = 0.025

model = Doc2Vec(size=vec_size,      # Dimensionality of the feature vectors
                window=8,            # The maximum distance 
                alpha=alpha,        # initial learning-rate
                min_alpha=0.00025,  # min learning-rate
                min_count=1,        # Ignores all words with total frequency lower than this.
                dm =1)              # pv-dm
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python2.7/dist-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved


Test

In [17]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
print("tokenize", test_data)
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('2')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs[1])

('tokenize', ['i', 'love', 'chatbots'])
('V1_infer', array([-1.7314958e-03,  1.5924233e-03, -1.0928866e-03, -5.3800328e-04,
       -2.3573051e-04,  6.4120727e-04, -1.4853590e-03,  5.4487732e-04,
       -8.7016444e-05, -5.9059850e-04,  1.6112410e-03,  2.0144519e-03,
        1.2877283e-03,  2.2541783e-03, -3.9156963e-04, -1.4925409e-03,
        2.0071269e-04, -1.0737969e-03,  8.5406611e-04, -9.4239152e-04,
        1.0105025e-03,  7.8221390e-05,  1.4437259e-03, -1.4325771e-03,
       -1.1917965e-03, -1.9226096e-03, -2.4465437e-03, -4.3783773e-04,
       -3.8964557e-04,  2.2069211e-03,  8.7877415e-04,  9.9919538e-04,
       -1.0577611e-03,  9.9718850e-04, -1.5652234e-03, -3.3169243e-04,
        1.4609672e-03, -1.2682481e-03,  1.0341750e-04,  7.3453598e-04,
       -6.4810170e-05,  1.0234307e-03,  3.8699008e-04, -2.2547296e-03,
       -3.3546097e-04, -4.8718162e-04,  2.0777616e-03,  1.9795161e-03,
       -1.2137505e-03,  2.2403020e-03,  4.2193072e-04,  6.6149246e-04,
       -4.3167357e-04,  

In [18]:
doc_vectorizer = Doc2Vec.load('d2v.model')

In [19]:
print(doc_vectorizer.wv.most_similar('machine'))

[('love', 0.9590880870819092), ('.', 0.9577491283416748), ('i', 0.9560537934303284), ('awesome', 0.9478388428688049), ('well', 0.9471067190170288), ('learning', 0.9448804259300232), ('amagingly', 0.9391720294952393), ('chat', 0.9360643625259399), ('its', 0.9340161085128784), ('they', 0.9301944375038147)]


# apply to gogosing data

In [20]:
num_cId = go_data['cId'].value_counts()
num_cId = pd.DataFrame(num_cId)
print(num_cId)

                      cId
네이버 페이 구매자          52539
rkdldk5**             475
yeji04**              449
elqxhs**              339
dnv**                 314
rladmswl9708**        308
wlgus5**              305
pp**                  301
wertyuio9**           292
sumin12**             278
dldmsql10**           276
kma05**               271
dream3**              245
rudgml72**            244
soa12**               237
kimhani3**            210
45216058**            209
sjj44**               197
rlaqhfk5**            193
tladnjs2**            191
tpwls12**             190
jy03**                189
egll**                188
jeje05**              187
bha11**               186
hooming**             186
tofha40**             186
ds357**               185
miliastla**           183
phr9101**             183
...                   ...
유민지                     1
461301292**             1
gook**                  1
byeolttar**             1
1698601763758607**      1
tjddlaks1**             1
eofhdrk**   

In [21]:
cIds = num_cId.drop(num_cId.index[0])  #drop naverpay
cIds = cIds[cIds['cId']!=1]  #drop count1
cIds_list = cIds.index.tolist()
len(cIds_list)

92802

In [22]:
df=(go_data.loc[go_data['cId']=='psbp**', ['cId','desc']])
print df['desc']

43527                                                                                                                                                                        조금 긴 슬랙스를 찾다가 고고싱 신상에 올라왔길래 바로 주문했어요평소 26입는데 s사이즈 조금 헐렁한데 괜찮아요근데 이렇게 얇은 재질인줄은 몰랐는데.. 여름에도 입을 수 있겠어요~~
43529                                                                                                                                                                                                                    데일리로 입을 흰 블라우스 찾다가 샀어요!저렴한 가격에 잘 산거 같아요 스판도 좋아서 불편하지않아요:)
43530                                                                                                                                                                                                                           흰색 나시가 필요해서 다른 옷 살때 나시도 같이 샀어요~이름처럼 스판 쫀쫀해서 좋아요ㅎㅎㅎ
102222                                                                                                                                                                         

In [23]:
len(df['desc'])

56

In [24]:
def tokenizer_twitter_morphs(doc):
    tagged_data = [TaggedDocument(words= twitter.morphs(doc), tags=[str(i)]) for i, _d in enumerate(df['desc'])]
    return tagged_data

In [25]:
def training_pvdm(tagged_data):
    max_epochs = 100
    vec_size = 20
    alpha = 0.025
    
    model = Doc2Vec(size=vec_size,      # Dimensionality of the feature vectors
                    window=8,            # The maximum distance 
                    alpha=alpha,        # initial learning-rate
                    min_alpha=0.00025,  # min learning-rate
                    min_count=1,        # Ignores all words with total frequency lower than this.
                    dm =1)              # pv-dm

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        #print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha


    print("Model Saved")
    
    return model

In [26]:
def similarity(model):
    sm = []
    for i in range(len(model.docvecs)-1):
        for j in range(i+1, len(model.docvecs)):
            sm.append(model.docvecs.similarity(i,j))
        
    print str('최대값: ') + str(max(sm))
    print str('상위25%: ') + str(np.percentile(sm,75))
    print str('중간값: ') + str(np.percentile(sm,50))
    print str('상위75%: ') + str(np.percentile(sm,25))
    print str('평균: ') + str(np.mean(sm))
    
    return sm

In [27]:
tagged_df = tokenizer_twitter_morphs('psbp**')
model = training_pvdm(tagged_df)
sm = similarity(model)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.8328396897953004
상위25%: 0.46729656940497166
중간값: 0.3361999315199555
상위75%: 0.1924911702966979
평균: 0.325525812002637


In [28]:
similar_doc = model.docvecs.most_similar('1')
print similar_doc

[('2', 0.6264384984970093), ('38', 0.6127101182937622), ('41', 0.6083108186721802), ('33', 0.5980275869369507), ('46', 0.5358717441558838), ('51', 0.5145502090454102), ('47', 0.5115160942077637), ('30', 0.49712538719177246), ('23', 0.4833873510360718), ('7', 0.45428138971328735)]


In [29]:
 # 소수점 아래 3자리까지 나타내기, 4째자리에서 반올림
def short_float(val):
    value = float("{:.4f}".format(val))
    return value


In [30]:
def print_want_val(sm_list,used_func):
    val=(used_func(sm_list))
    val=short_float(val)
    
    return val

In [32]:
max_list=[]
mid_list=[]
q1_list=[]
q3_list=[]

for i in range(len(cIds_list)):
    print str(i+1) + str('번째 리뷰어: ')+ str(cIds_list[i].encode('utf-8'))
    
    tagged_df = tokenizer_twitter_morphs(cIds_list[i])
    model = training_pvdm(tagged_df)
    sm = similarity(model)
    
    print str('most_similar 1')
    similar_doc = model.docvecs.most_similar('1')
    print similar_doc
    
    result_max= print_want_val(sm, lambda x: np.percentile(sm,100)) # 최대값 반환 
    max_list.append(result_max)
       
    result_q3 = print_want_val(sm, lambda x: np.percentile(sm,75)) #상위 25%     
    q3_list.append(result_q3)
    
    result_mid = print_want_val(sm, lambda x: np.percentile(sm,50)) # 중간값 반환 
    mid_list.append(result_mid)
    
    result_q1 = print_want_val(sm, lambda x: np.percentile(sm,25)) # 상위 75%
    q1_list.append(result_q1)

    print("\n")

1번째 리뷰어: rkdldk5**


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.9940013191061351
상위25%: 0.9868330268671966
중간값: 0.983641191988424
상위75%: 0.9802786866962716
평균: 0.9832401721093431
most_similar 1
[('2', 0.9919551610946655), ('47', 0.9905040264129639), ('46', 0.9904120564460754), ('33', 0.9900349974632263), ('30', 0.989831805229187), ('38', 0.989380419254303), ('7', 0.9886098504066467), ('51', 0.9880099296569824), ('42', 0.9869129657745361), ('44', 0.9868298172950745)]


2번째 리뷰어: yeji04**
Model Saved
최대값: 0.9940058890414196
상위25%: 0.9860885349836656
중간값: 0.9828820168645049
상위75%: 0.9790427539866111
평균: 0.9822346986422781
most_similar 1
[('2', 0.9916443824768066), ('33', 0.9895825386047363), ('47', 0.9895435571670532), ('46', 0.9894728660583496), ('30', 0.9893667101860046), ('7', 0.9878077507019043), ('22', 0.9871982336044312), ('54', 0.9868810772895813), ('51', 0.9867919683456421), ('24', 0.9866002798080444)]


3번째 리뷰어: elqxhs**
Model Saved
최대값: 0.8359327498073644
상위25%: 0.4737683784839018
중간값: 0.35240088495325755
상위75%: 0.207532859

Model Saved
최대값: 0.9948041678641655
상위25%: 0.9862175596092612
중간값: 0.9830367168392831
상위75%: 0.9794468841120263
평균: 0.9826999786247673
most_similar 1
[('2', 0.9904077053070068), ('38', 0.989880383014679), ('46', 0.9893120527267456), ('30', 0.9886497259140015), ('7', 0.987722635269165), ('42', 0.9876482486724854), ('33', 0.9875980615615845), ('35', 0.9865823984146118), ('51', 0.9865493774414062), ('44', 0.9861608743667603)]


21번째 리뷰어: jy03**
Model Saved
최대값: 0.9944074424781726
상위25%: 0.9868783635252725
중간값: 0.9836124116428402
상위75%: 0.9801950950496283
평균: 0.9832076253649035
most_similar 1
[('2', 0.9921656847000122), ('47', 0.9906222820281982), ('33', 0.9904789924621582), ('46', 0.9904143810272217), ('38', 0.9897408485412598), ('30', 0.9892328381538391), ('7', 0.9884779453277588), ('51', 0.9883819818496704), ('42', 0.9873462915420532), ('44', 0.9861708879470825)]


22번째 리뷰어: egll**
Model Saved
최대값: 0.8669212257562029
상위25%: 0.49473019245699845
중간값: 0.37400908953219225
상위75%: 0.237565496

Model Saved
최대값: 0.8133802242920235
상위25%: 0.4498672009978274
중간값: 0.315354836162672
상위75%: 0.1797552897017122
평균: 0.30748444897039967
most_similar 1
[('2', 0.6588936448097229), ('38', 0.5896185636520386), ('46', 0.5726125240325928), ('41', 0.5694286823272705), ('30', 0.5250650644302368), ('33', 0.5226466655731201), ('47', 0.4733697175979614), ('51', 0.46274930238723755), ('23', 0.45602911710739136), ('42', 0.45039990544319153)]


40번째 리뷰어: dekuu12**
Model Saved
최대값: 0.9948149912783697
상위25%: 0.987047797297989
중간값: 0.9838407737436715
상위75%: 0.9806284819709776
평균: 0.9835441227783709
most_similar 1
[('2', 0.9923550486564636), ('46', 0.9912378787994385), ('33', 0.9908870458602905), ('30', 0.9906620383262634), ('47', 0.9905019402503967), ('38', 0.9904834032058716), ('51', 0.9895406365394592), ('42', 0.9887716770172119), ('7', 0.9884809255599976), ('41', 0.9883087873458862)]


41번째 리뷰어: jjh86**
Model Saved
최대값: 0.9947043008152217
상위25%: 0.9871332797946982
중간값: 0.9840586092361745
상위75%: 0.98

...

Model Saved
최대값: 0.8185872662063324
상위25%: 0.4349821853754624
중간값: 0.2946530454876779
상위75%: 0.16775325653953047
평균: 0.2932456451522531
most_similar 1
[('2', 0.6230528354644775), ('38', 0.6003835797309875), ('33', 0.5734909772872925), ('41', 0.5501254796981812), ('46', 0.548061728477478), ('47', 0.5076574087142944), ('30', 0.495209664106369), ('35', 0.4336613416671753), ('3', 0.42758238315582275), ('23', 0.42710429430007935)]


952번째 리뷰어: apfhd**
Model Saved
최대값: 0.8002964428810415
상위25%: 0.45460498730579046
중간값: 0.3274751536494055
상위75%: 0.1927823032281215
평균: 0.3190314916706959
most_similar 1
[('2', 0.6751188039779663), ('38', 0.6129027605056763), ('41', 0.5907098054885864), ('46', 0.5704212784767151), ('30', 0.5338178277015686), ('33', 0.5190855860710144), ('47', 0.4995875060558319), ('23', 0.4778972566127777), ('42', 0.4719063639640808), ('3', 0.46958887577056885)]


953번째 리뷰어: thgml**
Model Saved
최대값: 0.850514871721283
상위25%: 0.4476110659787212
중간값: 0.32270892995662925
상위75%: 0.17

Model Saved
최대값: 0.9940594433697683
상위25%: 0.9863805544279616
중간값: 0.982946876890283
상위75%: 0.9793157952834746
평균: 0.9824997377673065
most_similar 1
[('2', 0.9919079542160034), ('33', 0.9902210235595703), ('47', 0.9899476766586304), ('46', 0.9898415207862854), ('30', 0.9888194799423218), ('38', 0.9887391328811646), ('7', 0.9882432222366333), ('51', 0.9879739880561829), ('42', 0.9859050512313843), ('54', 0.985154926776886)]


971번째 리뷰어: shy18**
Model Saved
최대값: 0.9945596501899984
상위25%: 0.9865542644668783
중간값: 0.9836692938611846
상위75%: 0.9801683046777279
평균: 0.983236756369035
most_similar 1
[('2', 0.9922103881835938), ('30', 0.9914348721504211), ('38', 0.9909133911132812), ('46', 0.9901624321937561), ('33', 0.9898704886436462), ('22', 0.9895618557929993), ('47', 0.9893538951873779), ('42', 0.9888612627983093), ('51', 0.9882106781005859), ('7', 0.9880962371826172)]


972번째 리뷰어: ksm9506**
Model Saved
최대값: 0.9944220724919828
상위25%: 0.9862417532083411
중간값: 0.9830367952644857
상위75%: 0.979650

Model Saved
최대값: 0.9950521309569773
상위25%: 0.9863357536538611
중간값: 0.9832473433118409
상위75%: 0.9794617315443825
평균: 0.9827777004116249
most_similar 1
[('38', 0.9918289184570312), ('2', 0.9915779829025269), ('46', 0.9899008274078369), ('33', 0.989870011806488), ('30', 0.9898695945739746), ('47', 0.9892434477806091), ('23', 0.9887282252311707), ('42', 0.9882670640945435), ('51', 0.9878902435302734), ('7', 0.9877358078956604)]


990번째 리뷰어: bora0142**
Model Saved
최대값: 0.9945214126214459
상위25%: 0.9866149290950297
중간값: 0.9835469720385459
상위75%: 0.9802916246237163
평균: 0.9831900209684118
most_similar 1
[('2', 0.9922084212303162), ('38', 0.9913761019706726), ('46', 0.9912995100021362), ('33', 0.9910658001899719), ('30', 0.9906536340713501), ('47', 0.9902434349060059), ('51', 0.9895685315132141), ('41', 0.9892674684524536), ('42', 0.9891067743301392), ('7', 0.988568902015686)]


991번째 리뷰어: mtkfk**
Model Saved
최대값: 0.795282524059936
상위25%: 0.4359849501042594
중간값: 0.2985920548954427
상위75%: 0.16759

Model Saved
최대값: 0.9951326179988323
상위25%: 0.9863682929709311
중간값: 0.9832461497346863
상위75%: 0.9797737081638982
평균: 0.9829334920510687
most_similar 1
[('30', 0.9914524555206299), ('2', 0.9912422299385071), ('33', 0.9903745651245117), ('38', 0.9902279376983643), ('46', 0.9898200035095215), ('42', 0.9889577627182007), ('7', 0.9878689646720886), ('51', 0.9876182079315186), ('54', 0.9874957203865051), ('47', 0.9871004819869995)]


1009번째 리뷰어: youmy10**
Model Saved
최대값: 0.9945801353874243
상위25%: 0.9869369547267848
중간값: 0.9837428581729608
상위75%: 0.9805936195019685
평균: 0.983497954475262
most_similar 1
[('2', 0.9924959540367126), ('38', 0.9914520978927612), ('30', 0.991174578666687), ('46', 0.9910863637924194), ('33', 0.9910137057304382), ('47', 0.9901705980300903), ('51', 0.9897875785827637), ('41', 0.9895628094673157), ('42', 0.9889215230941772), ('7', 0.9886353015899658)]


1010번째 리뷰어: lovee**
Model Saved
최대값: 0.8523577285198585
상위25%: 0.4830249676704351
중간값: 0.3591469115042536
상위75%: 0.223

KeyboardInterrupt: 

In [33]:
print(len(max_list))

1019
